In [ ]:
from encode_query import get_top_k, compute_scores

In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


def display_images(image_paths, priority_index = [], title = ""):
    """
    Displays a list of images in a single column.

    Parameters:
    image_paths (list): List of image file paths to be displayed.
    title (str): The title for the plot.
    """
    num_images = len(image_paths)

    prioritized_images = [image_paths[i] for i in priority_index]
    remain_images = [image_paths[i] for i in range(len(image_paths)) if i not in priority_index]
    
    image_paths = prioritized_images + remain_images

    # Create a figure with num_images rows and 1 column
    fig, axes = plt.subplots(num_images, 1, figsize=(6, num_images * 4))

    # Add a title to the entire figure
    fig.suptitle(title, fontsize=16)

    # Ensure axes is always a list (handles single image case)
    if num_images == 1:
        axes = [axes]
    
    index = 0
    # Iterate through the list of image paths and display each image
    for ax, img_path in zip(axes, image_paths):
        img = mpimg.imread(img_path)
        ax.imshow(img)
        ax.set_title(f"Index: {index}, {img_path.split('/')[-1].split('.')[0]}", fontsize=10)  # Set title to each image
        # ax.set_title(f"Index: {index}", fontsize=10)
        ax.axis('off')  # Hide axes
        index += 1
    
    plt.tight_layout(rect=[0, 0, 1, 0.97])  # Adjust layout to fit title
    plt.show()

In [5]:
with open("/mnt/mmlab2024/datasets/thi/key_index.txt", 'r') as r:
  key_f = r.read().split('\n')

In [3]:
import pathlib
def get_full_path_key_f(key_index):
  infor = key_index.split('_')

  jpg_path = pathlib.Path( f'/mnt/mmlab2024/nthy/Video/Videos_{infor[0]}/video/{infor[0]}_{infor[1]}/{infor[0]}_{infor[1]}_{infor[2]}.jpg')
  png_path = pathlib.Path( f'/mnt/mmlab2024/nthy/Video/Videos_{infor[0]}/video/{infor[0]}_{infor[1]}/{infor[0]}_{infor[1]}_{infor[2]}.png')

  return str(jpg_path) if jpg_path.exists() else str(png_path)


def infer_display(query, k = 100):
  top_probs, top_labels = get_top_k(query, k)
  selected_key_f = [get_full_path_key_f(key_f[i]) for i in top_labels]
  display_images(selected_key_f)
  return top_probs, top_labels

In [4]:
import pandas as pd

def export_csv(selected_indices, output_file, priority_index=[]):
  frame_infor = [key_f[i].split('_') for i in selected_indices]
  video_names = [f"{i[0]}_{i[1]}" for i in frame_infor]
  frame_index_in_video = [f"{i[2]}" for i in frame_infor]

  prioritized_videos = [video_names[i] for i in priority_index]
  remain_videos = [video_names[i] for i in range(len(video_names)) if i not in priority_index]

  prioritized_frame_idx = [frame_index_in_video[i] for i in priority_index]
  remain_frame_idx = [frame_index_in_video[i] for i in range(len(frame_index_in_video)) if i not in priority_index]
  
  final_video_names = prioritized_videos + remain_videos
  final_frame_idx_in_video = prioritized_frame_idx + remain_frame_idx

  final_video_names = final_video_names[0:100]
  final_frame_idx_in_video = final_frame_idx_in_video[0:100]
  
  if 'qa' in output_file.split('/')[-1]:
    df = pd.DataFrame.from_dict({"video_name" : final_video_names, "index" : final_frame_idx_in_video, "answer" : [""] * len(final_video_names)})
  else:
    df = pd.DataFrame.from_dict({"video_name" : final_video_names, "index" : final_frame_idx_in_video})
  # print(final_video_names)
  # print(final_frame_idx_in_video)
  
  df.to_csv(output_file, header=False, index=False)
    

## RUN from here

In [13]:
query = "A black boat can run on ice. It is propelled by a fan engine on top that blows air backwards. This boat is a rescue vehicle to assist victims who have fallen into an ice lake."
out_put_file = "/mnt/mmlab2024/datasets/thi/test_submit/query-1-kis.csv"

In [14]:
top_probs, top_labels = None, None
prioritized_list = []

In [ ]:
top_probs, top_labels = infer_display(query, k = 100)

## Pick and check

In [17]:
prioritized_list = [12, 17, 19, 22, 23, 28]

In [ ]:
selected_key_f = [get_full_path_key_f(key_f[i]) for i in top_labels]
display_images(selected_key_f, prioritized_list)

In [20]:
export_csv(top_labels, out_put_file, priority_index=prioritized_list)